In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
!pip install ../input/ensemble-boxes/ensemble_boxes-1.0.7-py3-none-any.whl  --no-deps
flips=[False,False,False,False]
CONF      = 0.2
IOU       = 0.50
from ensemble_boxes import *
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'

sizes=[4544,6400]
flips=[False,False,False,False]
CONF      = 0.2
IOU       = 0.50
AUGMENT   = False
MIN_SZ    = 4

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [ ]:
pwd

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/
!mkdir -p /root/mylibs/
!cp -r '../input/yolov5-lib-ds/models' /root/mylibs/
sys.path.insert(0,'/root/mylibs/')
# ../input/yolov5-lib-ds/models/experimental.py
sys.path.append("../input/yolov5-lib-ds/")
from models.experimental import attempt_load

In [ ]:

m1 = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
                       path='../input/yolov5s6/f2_sub2.pt',
                       source='local',
                       force_reload=True)  # local repo
m1.conf = 0.1


In [ ]:

m2 = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
                       path='../input/yolov5s6/yolov5s6_5.pt',
                       source='local',
                       force_reload=True)  # local repo
m2.conf = 0.1

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()  

# Run code

In [ ]:
def get_bboxs_yolo(results, IOU=0.25):
    anno = ''
    pds = results.pandas().xyxy[0]
    is_empty = pds.shape[0]
    if is_empty == 0:
        return ''
    for idx, row in pds.iterrows():
        if row.confidence > IOU:
            anno += f'{row.confidence} {int(row.xmin)} {int(row.ymin)} {int(row.xmax-row.xmin)} {int(row.ymax-row.ymin)} '
    return anno.strip(' ')        

In [ ]:
def box_scale(xmin, ymin, xmax, ymax, rate):
    x_nmin = xmin+(xmax-xmin)*rate
    y_nmin = ymin+(ymax-ymin)*rate
    x_nmax = xmax-(xmax-xmin)*rate
    y_nmax = ymax-(ymax-ymin)*rate
    return x_nmin,y_nmin,x_nmax,y_nmax

In [ ]:
def wbf_one_img_result(result_yolo1, result_yolo2, IOU_WBF=0.2):
    boxes_list = [[],[]]
    scores_list = [[], []]
    labels_list = [[], []]
    weights = [1, 1]
    iou_thr = 0.50
    skip_box_thr = 0.3
    #sigma = 0.1
    for idx1, row1 in result_yolo1.pandas().xyxy[0].iterrows():
        if row1.confidence > 0.1:
            boxes_list[0].append([row1.xmin/1280, row1.ymin/720, row1.xmax/1280, row1.ymax/720])
            scores_list[0].append(row1.confidence)
            labels_list[0].append(0)
    for idx2, row2 in result_yolo2.pandas().xyxy[0].iterrows():
        if row2.confidence > 0.1:
            boxes_list[1].append([row2.xmin/1280, row2.ymin/720, row2.xmax/1280, row2.ymax/720])
            scores_list[1].append(row2.confidence)
            labels_list[1].append(0)
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    length = len(scores)
    res = ''
    for idx in range(length):
        conf = scores[idx]
        xmin, ymin, xmax, ymax = boxes[idx]
        xnmin,ynmin,xnmax,ynmax = box_scale(xmin, ymin, xmax, ymax, 0.045)
        if conf > 0.3:
            res += f'{conf} {xnmin*1280} {ynmin*720} {(xnmax-xnmin)*1280} {(ynmax-ynmin)*720} '
    return res.strip(' ')

In [ ]:
for (pixel_array, df_pred) in iter_test:  # iterate through all test set images
    
    result_yolo1 = m1(pixel_array, size=9000, augment=True)
    #s_yolo  = get_bboxs_yolo(result_yolo)
    
    result_yolo2 = m2(pixel_array, size=9000, augment=True)
    #s_cbnet = get_bboxs_cbnet(result_cbnet)
    df_pred['annotations'] = wbf_one_img_result(result_yolo1, result_yolo2, IOU_WBF=0.2)
    env.predict(df_pred)

# Run Test